Hoja de trabajo 2

Realice una simulación de una sucursal de una cadena de supermercados, utilizando la librería Simpy de Python o utilizando el método de eventos discretos.

En una sucursal de una cadena de supermercados, llegan los clientes a un sistema con M cajas y con un proceso de Poisson a razón de 
 (>0) clientes por hora en promedio. Al llegar un cliente, se forma en una fila en una caja (selecciona la caja que tenga menos personas o en caso que el menor número de personas haya más de una caja, selecciona cualquiera entre esas cajas).  Cada cajero despacha al cliente en un tiempo que tiene una distribución exponencial con parámetro 
 (>0) clientes por hora.

Calcular: 

1. El tiempo promedio de un cliente en la cola.

2. Número de clientes en la cola en promedio.

3. Grado o factor de utilización de cada cajero.



Cajas (Servidores): Representan las cajas de la sucursal y servirán a los clientes.

Clientes: Llegan al sistema y se colocan en la caja con menos personas en fila.

Proceso de llegada de clientes: Los clientes llegan siguiendo un proceso de Poisson.

In [2]:
import simpy
import random

# Parámetros
LAMBDA = 10  # Clientes por hora
M = 3  # Cantidad de cajas
SIMULATION_TIME = 10  # Tiempo de simulación en horas

# Variables para almacenar datos
total_time_in_queue = 0
total_customers = 0
queue_lengths = []

def customer(env, server, i):
    global total_time_in_queue, total_customers
    arrival_time = env.now
    total_customers += 1
    
    with server.request() as req:
        yield req
        queue_length = len(server.queue)
        queue_lengths.append(queue_length)
        wait_time = env.now - arrival_time
        total_time_in_queue += wait_time
        service_time = random.expovariate(LAMBDA)
        yield env.timeout(service_time)

def setup(env, num_servers, rate):
    servers = [simpy.Resource(env) for _ in range(num_servers)]
    
    while True:
        yield env.timeout(random.expovariate(rate))
        env.process(customer(env, min(servers, key=lambda s: len(s.queue)), rate))

env = simpy.Environment()
env.process(setup(env, M, LAMBDA))
env.run(until=SIMULATION_TIME*60)  # Convertir horas a minutos para simulación

# Cálculos
average_time_in_queue = total_time_in_queue / total_customers
average_queue_length = sum(queue_lengths) / len(queue_lengths)
utilization = LAMBDA * average_time_in_queue / M

print(f"Tiempo promedio de un cliente en la cola: {average_time_in_queue:.2f} minutos")
print(f"Número de clientes en la cola en promedio: {average_queue_length:.2f}")
print(f"Grado o factor de utilización de cada cajero: {utilization:.2f}")



Tiempo promedio de un cliente en la cola: 0.05 minutos
Número de clientes en la cola en promedio: 0.01
Grado o factor de utilización de cada cajero: 0.15


Proceso de Poisson:
El proceso de Poisson es una forma de modelar eventos que ocurren de manera aleatoria a lo largo del tiempo, donde la probabilidad de que ocurra más de un evento en un intervalo de tiempo infinitesimalmente pequeño es prácticamente cero. En esta simulación, la llegada de clientes al supermercado sigue un proceso de Poisson. Esto significa que en cualquier momento, un cliente podría llegar, pero la probabilidad de que dos o más clientes lleguen al mismo instante es extremadamente baja.

En términos prácticos, usamos la distribución exponencial (que está relacionada con el proceso de Poisson) para determinar el tiempo entre llegadas sucesivas. Si se dice que hay λ clientes que llegan por hora en promedio (la tasa del proceso de Poisson), el tiempo entre las llegadas sigue una distribución exponencial con un parámetro λ.

En el código, esto se refleja en la línea:

**yield env.timeout(random.expovariate(rate))**

Donde rate es λ y random.expovariate(rate) genera tiempos aleatorios entre llegadas basados en la distribución exponencial.

Distribución Exponencial:
Además de modelar el tiempo entre llegadas, la distribución exponencial se usa para modelar el tiempo de servicio del cajero. Esto se basa en la suposición de que el tiempo que un cajero tarda en atender a un cliente es variable y sigue esta distribución.

Esto es modelado en el código con la línea:

**service_time = random.expovariate(LAMBDA)**
Esto significa que el tiempo que toma servir a un cliente es una variable aleatoria con una distribución exponencial, nuevamente con el parámetro λ.

Sistema de Cola:
El comportamiento de la cola (los clientes esperando ser atendidos) es el núcleo de esta simulación. En esta simulación, cuando un cliente llega, él o ella selecciona la caja con la fila más corta. Si hay múltiples cajas con la misma longitud de fila más corta, el cliente elige aleatoriamente una de esas cajas. Esto es modelado por:

**min(servers, key=lambda s: len(s.queue))**

Donde servers es la lista de cajas y se selecciona la caja con la fila más corta.